In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,mean_absolute_percentage_error
import catboost
import matplotlib.pyplot as plt
from src import subfunc as sf

In [2]:
DATA_PATH = '../../data/'
X_train = pd.read_csv(f'{DATA_PATH}X_train.csv',sep='\t', index_col='index')
X_test = pd.read_csv(f'{DATA_PATH}X_test.csv', sep='\t', index_col='index')
y_train = pd.read_csv(f'{DATA_PATH}y_train.csv',sep='\t', index_col='index')
y_test = pd.read_csv(f'{DATA_PATH}y_test.csv',sep='\t', index_col='index')

In [3]:
target_columns = y_train.columns.to_list()

* На предсказание целевых переменных сильно влияют показатели последнего хим анализа, что видно из корреляции. Мы будем предсказывать и определять их как признаки для обучающей и тестовой выборки


In [4]:
future_columns = [column for column in X_train.columns if 'последний' in column]

y_train[future_columns] = X_train[future_columns]
X_train = X_train.drop(columns=future_columns)
y_test[future_columns] = X_test[future_columns]
X_test = X_test.drop(columns=future_columns)

In [5]:
future_columns

['химсталь последний Al',
 'химсталь последний C',
 'химсталь последний Ca',
 'химсталь последний Cr',
 'химсталь последний Cu',
 'химсталь последний Mn',
 'химсталь последний Mo',
 'химсталь последний N',
 'химсталь последний Ni',
 'химсталь последний P',
 'химсталь последний S',
 'химсталь последний Si',
 'химсталь последний Ti',
 'химсталь последний V',
 'химшлак последний FeO',
 'химшлак последний MgO',
 'химшлак последний MnO']

In [6]:
X_train, X_test = sf.fill_future_columns(X_train, X_test, y_train, y_test, future_columns)
y_train = y_train.drop(columns=future_columns)
y_test = y_test.drop(columns=future_columns)

for column химсталь последний Cu: 0.9752326992661476
for column химсталь последний Ni: 0.9565665461446651
for column химсталь последний P: 0.8958044222305857
for column химсталь последний Mo: 0.8566425477812905
for column химсталь последний N: 0.5014055737845802


* CatBoostRegressor

In [7]:
models = {}
target_columns_list = target_columns.copy()
while len(target_columns_list) > 0:
    prediction_result = {}
    for column in target_columns_list:
        cur_eval_data = catboost.Pool(X_test, y_test[column])
        cur_model = catboost.CatBoostRegressor(iterations=5000,
                            learning_rate=0.05,
                            task_type='GPU',
                            loss_function='MAE',
                            random_seed= 42)
        cur_model.fit(X_train, y_train[column],
                      use_best_model=True,
                      early_stopping_rounds=100,
                      eval_set=cur_eval_data,
                      silent=True)
        prediction_result[column] = [cur_model.copy(), r2_score(
            y_pred=cur_model.predict(X_test), y_true=y_test[column])]

    best_r2 = -20
    best_target = ''
    for key in prediction_result.keys():
        if prediction_result[key][1] > best_r2:
            best_r2 = prediction_result[key][1]
            best_target = key

    X_train[best_target] = prediction_result[best_target][0].predict(X_train)
    X_test[best_target] = prediction_result[best_target][0].predict(X_test)
    models[best_target] = prediction_result[best_target][0]
    target_columns_list.remove(best_target)


In [8]:
models2 = {}
target_columns_list = target_columns.copy()

for column in target_columns_list:
    cur_eval_data = catboost.Pool(X_test, y_test[column])
    cur_model = catboost.CatBoostRegressor(iterations=5000,
                        learning_rate=0.05,
                        task_type='GPU',
                        loss_function='MAE',
                        random_seed= 42)
    cur_model.fit(X_train, y_train[column],
                  use_best_model=True,
                  early_stopping_rounds=100,
                  eval_set=cur_eval_data,
                  silent=True)

    models2[column] = cur_model.copy()

 * Предсказание, когда таргет переменные учавствуют в предсказании других таргет переменных

In [9]:
sf.get_report(models, target_columns, X_test, y_test)

,R2,MAE,MSE,MAPe
химшлак последний Al2O3,0.502823,0.465914,0.422492,10.161562
химшлак последний CaO,0.174481,2.579355,11.452506,4.731759
химшлак последний R,0.361938,0.139304,0.031829,5.818449
химшлак последний SiO2,0.131946,1.588209,4.214151,6.867264


 * Предсказание, когда таргет переменные не учавствуют в предсказании других таргет переменных

In [10]:
sf.get_report(models2, target_columns, X_test, y_test)

,R2,MAE,MSE,MAPe
химшлак последний Al2O3,0.501380,0.464661,0.423719,10.137905
химшлак последний CaO,0.177913,2.596930,11.404905,4.751353
химшлак последний R,0.289411,0.143745,0.035447,5.996757
химшлак последний SiO2,0.113165,1.606529,4.305327,6.957660


* GradientBoostingRegressor

In [11]:
gbr_models = {}
target_columns_list = target_columns.copy()
while len(target_columns_list) > 0:
    prediction_result = {}
    for column in target_columns_list:
        best_r2_model_score = -20
        best_r2_model = None
        for n_estimators in np.arange(250, 350, 50):
            for lr in np.arange(0.001, 0.1, 0.01):
                cur_model = GradientBoostingRegressor(n_estimators=n_estimators,
                                                      learning_rate=lr,
                                                      random_state=42
                                                      )
                cur_model.fit(X_train, y_train[column])
                pred = cur_model.predict(X_test)
                cur_r2 = r2_score(y_true=y_test[column], y_pred=pred)
                if cur_r2 > best_r2_model_score:
                    best_r2_model_score = cur_r2
                    best_r2_model = cur_model
        prediction_result[column] = [best_r2_model, best_r2_model_score]

    best_r2 = -20
    best_target = ''
    for key in prediction_result.keys():
        if prediction_result[key][1] > best_r2:
            best_r2 = prediction_result[key][1]
            best_target = key

    X_train[best_target] = prediction_result[best_target][0].predict(X_train)
    X_test[best_target] = prediction_result[best_target][0].predict(X_test)
    gbr_models[best_target] = prediction_result[best_target][0]
    target_columns_list.remove(best_target)

In [12]:
gbr_models2 = {}
target_columns_list = target_columns.copy()
prediction_result = {}

for column in target_columns_list:
    best_r2_model_score = -20
    best_r2_model = None
    for n_estimators in np.arange(250, 350, 50):
        for lr in np.arange(0.001, 0.1, 0.01):
            cur_model = GradientBoostingRegressor(n_estimators=n_estimators,
                                                  learning_rate=lr,
                                                  random_state=42
                                                  )
            cur_model.fit(X_train, y_train[column])
            pred = cur_model.predict(X_test)
            cur_r2 = r2_score(y_true=y_test[column], y_pred=pred)
            if cur_r2 > best_r2_model_score:
                best_r2_model_score = cur_r2
                best_r2_model = cur_model
    prediction_result[column] = [best_r2_model, best_r2_model_score]

    gbr_models2[column] = prediction_result[column][0]

 * Предсказание, когда таргет переменные учавствуют в предсказании других таргет переменных

In [13]:
sf.get_report(gbr_models, target_columns, X_test, y_test)

,R2,MAE,MSE,MAPe
химшлак последний Al2O3,0.472921,0.489016,0.447903,10.903660
химшлак последний CaO,-0.127569,3.149333,15.642885,5.622758
химшлак последний R,0.284008,0.149118,0.035717,6.242387
химшлак последний SiO2,0.069631,1.705198,4.516671,7.261876


 * Предсказание, когда таргет переменные не учавствуют в предсказании других таргет переменных

In [14]:
sf.get_report(gbr_models2, target_columns, X_test, y_test)

,R2,MAE,MSE,MAPe
химшлак последний Al2O3,0.490339,0.476304,0.433102,10.515466
химшлак последний CaO,0.131656,2.701544,12.046634,4.904466
химшлак последний R,0.296260,0.145291,0.035106,6.077345
химшлак последний SiO2,0.099676,1.813891,4.370815,7.773758


* LinearRegression

In [15]:
linear_models = {}
target_columns_list = target_columns.copy()
while len(target_columns_list) > 0:
    prediction_result = {}
    for column in target_columns_list:
        cur_model = LinearRegression()
        cur_model.fit(X_train, y_train[column])

        prediction_result[column] = [cur_model, r2_score(
            y_pred=cur_model.predict(X_test), y_true=y_test[column])]
    best_r2 = -20
    best_target = ''
    for key in prediction_result.keys():
        if prediction_result[key][1] > best_r2:
            best_r2 = prediction_result[key][1]
            best_target = key

    X_train[best_target] = prediction_result[best_target][0].predict(X_train)
    X_test[best_target] = prediction_result[best_target][0].predict(X_test)
    linear_models[best_target] = prediction_result[best_target][0]
    target_columns_list.remove(best_target)

In [16]:
linear_models2 = {}
target_columns_list = target_columns.copy()

for column in target_columns_list:
    cur_model = LinearRegression()
    cur_model.fit(X_train, y_train[column])
    linear_models2[column] = cur_model

 * Предсказание, когда таргет переменные учавствуют в предсказании других таргет переменных

In [17]:
sf.get_report(linear_models, target_columns, X_test, y_test)

,R2,MAE,MSE,MAPe
химшлак последний Al2O3,0.086146,0.640631,0.776578,14.176976
химшлак последний CaO,-0.319258,3.312788,18.302200,5.985589
химшлак последний R,0.142281,0.161920,0.042787,6.798154
химшлак последний SiO2,-0.828821,2.350044,8.878399,10.034826


 * Предсказание, когда таргет переменные не учавствуют в предсказании других таргет переменных

In [18]:
sf.get_report(linear_models2, target_columns, X_test, y_test)

,R2,MAE,MSE,MAPe
химшлак последний Al2O3,0.410492,0.508509,0.500954,11.209618
химшлак последний CaO,0.061238,2.801976,13.023549,5.078468
химшлак последний R,0.269410,0.148082,0.036445,6.203759
химшлак последний SiO2,0.017557,1.711039,4.769476,7.335545
